<a href="https://colab.research.google.com/github/JumpLusWu/CIS700/blob/master/%E2%80%9CHW1_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Git (0 pts)

Run the following to get the required files needed for this assignment. 

In [3]:
!git clone https://github.com/cis700/hw1-release.git
!mv hw1-release/dills/* .
!mv hw1-release hw1

Cloning into 'hw1-release'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 26 (delta 6), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [3]:
!ls

hw1	       q3_3_loader.p  q3_4_net.p     q3_6_loader.p  q3_7_net.p
q3_2_loader.p  q3_3_net.p     q3_5_loader.p  q3_6_net.p     sample_data
q3_2_net.p     q3_4_loader.p  q3_5_net.p     q3_7_loader.p


# Classical vs. Deep Learning Approach


## Question 1. Using Decision Trees to Classify Fashion-MNIST (10 pts)

**Understanding the Dataset**

The first step to machine learning is to understand the dataset that you're given. Therefore, familiarize yourself with the Fashion-MNIST dataset. We recommend reading the paper published about the dataset, which is detailed [here](https://arxiv.org/pdf/1708.07747.pdf).

**Q1a (2 pts):** Explain at a high level what the dataset is (i.e. what are the specifications of the input data and associated labels).

The input data comes from the pictures of fashion products and is preprocessed to have the same image size as the MINST dataset. It has ten different classes and is labeled using numbers.

**Loading the Data**

Since there's no built in dataloader for Fashion-MNIST built into sci-kit learn, we've created a dataloader that you may use. The first code snippet is to the install [wget](https://pypi.org/project/wget/) and sets up the directory structure for your data to go in.

In the second code snippet, we provide the function made by the authors of the fashion-MNIST dataset to extract the downloaded data.

**Q1b (3 pts):** Fill in the load_datasets function such that it returns the training data, training labels, testing data, and testing labels. In your report, indicate the dimensions of the training set and the testing set respectively. 

In [4]:
!pip install wget
!mkdir -p data/fashion

  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
import os.path
import wget
import pdb

## Function provided by the creators of Fashion-MNIST to unpack and load their data
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [0]:
# Fill in this method to load the Fashion-MNIST dataset
def load_datasets():
  base_URL = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"
  files = ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz", "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"]
  fashion_dir = "data/fashion"
  import wget
  base_URL = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"
  files = ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz", "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"]
  for i in range(4):
    wget.download(base_URL+files[i], 'data/fashion') # download the compressed data file
  train_images, train_labels =load_mnist('data/fashion', kind='train')
  test_images, test_labels=  load_mnist('data/fashion', kind='t10k')
  # Return the correct four variables
  return train_images, train_labels, test_images, test_labels

train_images, train_labels, test_images, test_labels = load_datasets()

In [0]:
print(train_labels
     )

[9 0 0 ... 3 0 5]


**Creating a Classical Classifier**

**Q1c (5 pts):** Use logistic regression to classify the Fashion-MNIST dataset. Note that because this is not sped up by a GPU this may take a few minutes to run. In your report, plot the learning curve for train and validation accuracy and report the final classification accuracy.



In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='lbfgs', multi_class='multinomial')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(train_images, train_labels)
Z = logreg.predict(test_images)
print (accuracy_score(test_labels, Z))

0.8412


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
print (accuracy_score(test_labels, Z))

0.8412


## Question 2. Using Deep Learning to Classify Fashion-MNIST (10 pts)

In this section you will also classify the Fashion-MNIST dataset, however, with a deep neural network instead.

**Downloading PyTorch**

Run the code snippet below to install PyTorch / required packages and set up the GPU.

In [0]:
## Required packages
!pip install tensorflow
!pip install scipy
!pip install numpy
!pip install Pillow
!pip install image

## Add any other packages you may need below

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

    100% |████████████████████████████████| 614.8MB 26kB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
    100% |████████████████████████████████| 2.0MB 14.0MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


cuda:0


**PyTorch Tensorboard Integration**

In this class we present the option of using tensorboard to visualize loss / accuracy plots and images. You may choose to use matplotlib instead, however, we recommend using tensorboard as it's becoming the industry standard for analyzing / debugging neural networks. 

Simply run the code snippets below in the order provided, and click on the link to open up the tensorboard dashboard (which should be blank right now!).

In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

In [74]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: http://34c81d04.ngrok.io


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [73]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: http://34c81d04.ngrok.io


In [0]:
# Use this object to help with tensorboard integration!
from hw1.helper import Logger
import numpy as np
# logger = Logger('./logs')
# info = { 'loss': 5, 'accuracy': 6 }
# step =0
# c = np.arange(1000)
# d = np.arange(2000,3000,1)
# for i in range(1000):
#         logger.scalar_summary('loss', d[i], step+1)
#         step=step+1


To see how to use Pytorch with tensorboard, check out [this](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/04-utils/tensorboard/main.py) github repo (specifically in main.py). Try plotting something using tensorboard to ensure that it's all working. For help, come to office hours or check out [this](https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab) link (where we got the tensorboard-colab integration).

**Important:** Note that we've changed the logging folder to "./logs" . Keep that in mind when using tensorboard.

**PyTorch Dataloaders **

The first step to deep learning is to create a dataloader. It's important to understand how a dataloader works so you can create a custom dataloader for datasets in the future. To understand more, we highly recommend PyTorch's tutorial on dataloaders detailed [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

**Q2a (2 pts):** Create a training and testing dataloader for the Fashion-MNIST dataset. Report the size of the training and testing sets respectively (it should match those from the previous question.)

*Hint:* Fashion-MNIST is a highly used dataset, therefore you do not have to create a whole dataloader from scratch. For some help, take a look at Torch's [dataset](https://pytorch.org/docs/stable/torchvision/datasets.html) and [dataloader](https://pytorch.org/docs/stable/data.html) page.

In [0]:
import torchvision
import torchvision.transforms as transforms
#import 
root = os.path.join('data','/fashion')
print(root)
fashion_train_dataset = torchvision.datasets.FashionMNIST(root, train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
fashion_test_dataset = torchvision.datasets.FashionMNIST(root, train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

/fashion
Processing...
Done!


**Creating the Neural Network Model**

PyTorch has a fantastic tutorial on creating a neural network model, detailed [here](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html).

**Q2b (2 pts):** Using knowledge from lecture and the tutorial from the link above, create a neural network model to classify the Fashion-MNIST dataset. In your report, detail your network architecture, explaining the choices that you made (i.e. depth of the neural network, activation functions, etc).

*NOTE:* You may **not** use convolutional layers in this model.

In [0]:
import torch.nn as nn
import torch.nn.functional as F
# fully connected neural network with one hidden layer
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size,num_classes)
    # need softmax?
    
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out
  
    
    

In [0]:
nn =Net(28*28,500,10)
params = list(nn.parameters())
print(len(params ))  # 4 layer's weight

4


In [0]:
print(nn)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


**Creating the Training Loop**

**Q2c (2 pts):** Create a training loop that goes over the entire Fashion-MNIST dataset once. You should tune all hyperparameters (i.e. batch-size, learning rate, etc). Use tensorboard so that in one graph, plot accuracy vs iterations, and in the other plot loss vs iterations (axis labels are not needed if you're using tensorboard). Report the final testing accuracy and loss.

In [0]:
! rm -r ./logs # clear the tensorboard

In [0]:
import time, datetime
import torch.nn as nn
import torch.nn.functional as F
# fully connected neural network with one hidden layer
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size,num_classes)
    # need softmax?
    
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

# for debug use
import pdb
# Use this object to help with tensorboard integration!
from hw1.helper import Logger
now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/run_{now}/')  # create different logger according time

# hyper-parameters
input_size = 28*28
hidden_size  = 500
num_classes = 10
batch_size  = 128
learning_rate = 0.001
num_epochs = 5
# load data
train_loader = torch.utils.data.DataLoader(fashion_train_dataset,batch_size = batch_size,shuffle =True)
test_loader = torch.utils.data.DataLoader(fashion_test_dataset,batch_size = batch_size,shuffle =False)


model  = Net(input_size, hidden_size,num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

# train the model
total_step =  len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    # move tensors  to the configured device
    
     
    #print(images.shape)
    # images input 128*1*28*28
    images = images.reshape(-1,28*28).to(device)  # 128*28*28
    
    labels = labels.to(device)
    
    # forward pass
    outputs = model(images)
    loss = criterion(outputs,labels)
    
    # backward and optimize
    optimizer.zero_grad() # clear optimizer
    loss.backward()
    optimizer.step()
    
    # compute accuracy
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()  # compare 128
    info = { 'loss': loss.item(), 'accuracy': accuracy.item() } #  need to use item, accuracy is the tensor>tensor(0.0781, device='cuda:0')
    # debug
    for tag, value in info.items():
      logger.scalar_summary(tag, value, epoch+1)
    
    
    
    


With softmax, why worse?

**Q2d (2 pts):** Create a testing loop that outputs the accuracy of your trained neural network on the test set. Report the evaluation accuracy of your network on this testing set.

*Hint:* Look at the torch.no_grad() function to evaluate your neural network without updating the weights.

In [0]:
# test the model
# don't need to compute gradients for memory efficiency
with torch.no_grad(): 
  correct  = 0
  total  = 0
  for images,labels in test_loader:
    images = images.reshape(-1,28*28).to(device) #why need one more dimension -1, because is the batch
    labels = labels.to(device)
    outputs = model(images)
   # pdb.set_trace()
    _,predicted = torch.max(outputs.data,1) # outputs.data will output two tensor, one is prob ,one is label
    total += labels.size(0) # size += 128
    correct += (predicted == labels).sum().item()  #.sum().item():because .sum is still a tensor use .item to get rid of cuda
    
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
    
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')
    
    
    

**Q2e (2 pts):** Compare the testing accuracy of your neural network vs. the decision tree clasifier you made in the previous part.

The test accuracy is a bit of higher compared to logistic regression. 87% vs 84%.

In [0]:
!rm -r logs/run_1551284730.0
!rm -r logs/run_1551284785.0

# Question 3. Deep Learning Diagnosis

### Q3.1.a MNIST control group (10 pts)

Using the same PyTorch approach as above, tune the hyperparameters for a neural net on the MNIST dataset.  Plot the learning curve and report its final accuracy in your writeup.  This neural net's performance will serve as the reference point (a "control group") for your analysis in parts 3.2, 3.4, and 3.7

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time, datetime
import pdb

# Tune hyperparameters here
num_epochs = 5
batch_size = 128
hidden_layer = 256
input_layer  =28*28
classes = 10
leaning_rate = 1e-3

# Load the MNIST dataset (images and labels, both train and test) into 2 DataLoaders
train_dataset = torchvision.datasets.MNIST(root='./data_mnist', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data_mnist', 
                                          train=False, 
                                          transform=transforms.ToTensor(),
                                          download = True)
# load data
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size,shuffle =True)
test_loader  = torch.utils.data.DataLoader(test_dataset,batch_size = batch_size,shuffle =False)

class Net(nn.Module):

    def __init__(self,input_layer ,hidden_layer,classes):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_layer ,hidden_layer)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_layer,classes)
        
    def forward(self, x):
      # TODO: define the forward pass
      out = self.fc1(x)
      out = self.relu(out)
      out = self.fc2(out)
      return out

# logger for tensorboard
now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/run_{now}/')  # create different logger according time
    
# TODO: complete the train loop
total_step = len(train_dataset)
model =  Net(input_layer,hidden_layer,classes).to(device)
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      input_image  = images.reshape(-1,28*28).to(device)
      
      labels  = labels.to(device)
     
    # forward pass
      outputs2 =  model(input_image)
      loss = criterion(outputs2,labels)
      
    # backward pass and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    # accuracy
      _, argmax = torch.max(outputs2,1)

      # accuracy  = (labels == argmax).mean()
      accuracy  = (labels == argmax.squeeze()).float().mean()
      # learing curve
      curve ={'loss':loss.item(),'accuracy':accuracy.item()}
      # add to summary
      for tag,value in curve.items():
        logger.scalar_summary(tag,value,i*(epoch+1)+1)
   

      
      
      
      
    # TODO: plot the learning curve.


In [0]:
# TODO: complete the test loop
with torch.no_grad(): 
  correct  = 0
  total  = 0
  for images,labels in test_loader:
    images = images.reshape(-1,28*28).to(device) #why need one more dimension -1, because is the batch
    labels = labels.to(device)
    outputs = model(images)
 
   # pdb.set_trace()
    _,predicted = torch.max(outputs.data,1) # outputs.data will output two tensor, one is prob ,one is label
    total += labels.size(0) # size += 128
    correct += (predicted == labels).sum().item()  #.sum().item():because .sum is still a tensor use .item to get rid of cuda
    
  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
    
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')
    



Accuracy of the network on the 10000 test images: 97.53 %


train_loss:0.11 train_accuracy:96%

### Q3.1.b CIFAR control group (10 pts)

Similar to the previous part, create a neural network, training, and testing loop and tune the hyperparameters for the CIFAR-10 dataset. Plot the learning curve and report its final accuracy in your writeup.  This neural net's performance will serve as the reference point (a "control group") for your analysis in parts 3.3, 3.5, and 3.6

**First try: use CNN**

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time, datetime
# Tune hyperparameters here

num_epochs = 80
num_classes = 10
batch_size = 128
learning_rate = 0.001
# logger for tensorboard
now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/run_{now}/')  # create different logger according time

# Load the MNIST dataset (images and labels, both train and test) into 2 DataLoaders
train_dataset = torchvision.datasets.CIFAR10(root='./data_cifar', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=False)

test_dataset = torchvision.datasets.CIFAR10(root='./data_cifar', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# data_loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle =False)

class ConvNet(nn.Module):

    def __init__(self,num_classes=10):
      super(ConvNet, self).__init__()
      self.layer1 = nn.Sequential(
          nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer2 = nn.Sequential(
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2))
      self.fc = nn.Linear(8*8*32, num_classes)  # 32/2/2 

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #pdb.set_trace()
        out = out.reshape(out.size(0), -1)  # out 128*32*8*8
        out = self.fc(out)
        return out
model = ConvNet(num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# TODO: complete the train loop
# Train the model
total_step = len(train_loader)
step = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
        _, argmax = torch.max(outputs,1)
        accuracy  = (labels == argmax.squeeze()).float().mean()  # the accuracy is the average of 128 images? or more
      
        info ={'loss':loss.item(),'accuracy':accuracy.item()}
        #pdb.set_trace()
        step = step+1
        logger.scalar_summary('loss',info['loss'],step)
        logger.scalar_summary('accuracy',info['accuracy'],step)
# TODO: complete the test loop
# TODO: plot the learne.

In [0]:
torch.save(model.state_dict(), 'model.ckpt')

In [0]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    # test in batch

Test Accuracy of the model on the 10000 test images: 83.346 %


For each question below, we will provide 3 PyTorch objects: a DataLoader, a neural network model, and an optimizer (you will load them using dill, a serialization library in Python).  For each question, please complete the following:

a.   Train the given neural net using the given DataLoader and the given optimizer.  Plot the learning curve of the network and report the final accuracy in your writeup.

b.   Identify whether the neural net seems to be training correctly.  If not, debug the issue (e.g. printing parameters, gradients, outputs) and report your methodology in your writeup. In addition, we highly recommend checking out [this](https://docs.google.com/document/d/11R5IiLMjddIWM2csfjFBZ__Adu0sY0zuVhbS3_5lLBQ/edit?usp=sharing) google doc with information about common failure modes for training deep neural networks.

c.   If you identify an issue with any of the 3 given objects, replace the buggy object with one of your own making.  Then retrain the neural net, plot the new learning curve, and report the improved accuracy in your writeup.  Note that in each of the following problems, exactly 1 object out of the DataLoader, the neural net, and the optimizer will be buggy.  You should not have to replace more than 1.

Run the code snippet below for some necessary packages / setup.

In [0]:
import torch
import torch.nn as nn
import dill
import torch.optim as optim
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Q3.2 MNIST diagnosis 1 (10 pts)

In [0]:
q_3_2_data_loader = dill.load( open( "q3_2_loader.p", "rb" ) )
train_loader, test_loader = q_3_2_data_loader()
q_3_2_net_loader = dill.load( open( "q3_2_net.p", "rb" ) )
net, optimizer = q_3_2_net_loader()

In [104]:
print(train_loader.batch_size)

100


From the tensorboard, it seems that the network was initailzed with zero mean and zero weight

In [105]:
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0003
    weight_decay: 0
)


In [36]:
num_epochs = 8
from hw1.helper import Logger
import time, datetime
import pdb
now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/run_{now}/') 

# loss 
criterion = nn.CrossEntropyLoss()

step = 0

# problem fix
torch.nn.init.xavier_uniform(net.fc1.weight)
torch.nn.init.xavier_uniform(net.fc2.weight)
torch.nn.init.xavier_uniform(net.fc3.weight)
torch.nn.init.xavier_uniform(net.fc4.weight)
torch.nn.init.xavier_uniform(net.fc5.weight)
model = net.to(device)


               
for i in range(num_epochs):
  for j,(images,labels)  in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    #pdb.set_trace()
    #forward pass
   
    #pdb.set_trace()
    outputs  = model(images)
    #pdb.set_trace()
    loss  = criterion(outputs,labels) # pos can't be reverted because need preprocessing for outputs
    
    # backward pass
    optimizer.zero_grad()
    #pdb.set_trace()
    loss.backward()
    #pdb.set_trace()
    optimizer.step()
    
    
    # record the accuracy and loss
    step = step +1
   # pdb.set_trace()
    _,predicted = torch.max(outputs,1)
    accuracy  =  (labels == predicted).float().mean()
    info  = {'loss':loss.item(),'accuracy':accuracy.item()}
    for tag, value in info.items():
      logger.scalar_summary(tag,value,step)
      # 2. Log values and gradients of the parameters (histogram summary)
    for tag, value in model.named_parameters():
        tag = tag.replace('.', '/')
        logger.histo_summary(tag, value.data.cpu().numpy(), step)
        logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step)

    # 3. Log training images (image summary)
    info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

    for tag, images in info.items():
        logger.image_summary(tag, images, step)
    if step %100 ==0:
      print('accuracy:{}'.format(accuracy.item()))
      
    
   
    
    
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


accuracy:0.9699999690055847
accuracy:0.9799999594688416
accuracy:1.0
accuracy:1.0
accuracy:1.0


In [199]:


# check test accuracy
with torch.no_grad():
  correct = 0
  total  = 0
  for images,labels in test_loader:
    images= images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    #pdb.set_trace()
    _,predicted  = torch.max(outputs.data,1)
    correct +=  (labels == predicted).sum().item()
    total += labels.size(0)
    
  print('accuracy of the network for testing data is:{}'.format(correct/total))
    
    
    
    



accuracy of the network for testing data is:0.1084


### Q3.3 CIFAR diagnosis 1 (10 pts)

**Diagnositic: It seems that the train data is not shuffled. To fix it, I shuffle the dataset.**

In [168]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: https://3c4c45c7.ngrok.io


In [211]:

q_3_3_data_loader = dill.load( open( "q3_3_loader.p", "rb" ) )
train_loader2, test_loader2 = q_3_3_data_loader()
q_3_3_net_loader = dill.load( open( "q3_3_net.p", "rb" ) )
net, optimizer = q_3_3_net_loader()


Files already downloaded and verified
Files already downloaded and verified


In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_loader2.dataset,
                                          batch_size = train_loader2.batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset=train_loader2.dataset,
                                          batch_size = train_loader2.batch_size,
                                          shuffle =False)

In [215]:
import pdb


for images, labels in train_loader:
 # pdb.set_trace()
  count= []
  for i in range(10):
    count.append(sum(labels==i).item())
    
  print(count)

[10, 8, 12, 10, 7, 10, 10, 9, 15, 9]
[10, 6, 9, 13, 14, 11, 9, 8, 13, 7]
[6, 12, 12, 14, 11, 9, 11, 6, 10, 9]
[13, 9, 7, 9, 10, 12, 12, 15, 2, 11]
[11, 12, 13, 6, 8, 11, 13, 7, 13, 6]
[12, 8, 9, 18, 11, 8, 4, 10, 9, 11]
[6, 11, 15, 11, 11, 9, 10, 9, 12, 6]
[11, 13, 9, 12, 13, 11, 2, 7, 12, 10]
[9, 14, 8, 10, 7, 12, 13, 10, 10, 7]
[7, 8, 8, 9, 14, 12, 11, 10, 7, 14]
[8, 11, 8, 12, 7, 9, 19, 6, 6, 14]
[15, 12, 7, 11, 11, 10, 8, 11, 10, 5]
[6, 15, 9, 8, 10, 12, 10, 5, 16, 9]
[18, 9, 5, 12, 11, 7, 11, 6, 9, 12]
[4, 8, 14, 5, 11, 10, 12, 14, 13, 9]
[20, 8, 11, 12, 5, 12, 7, 13, 8, 4]
[11, 7, 18, 8, 8, 13, 11, 9, 5, 10]
[10, 13, 14, 6, 5, 11, 13, 8, 8, 12]
[9, 8, 14, 12, 10, 11, 10, 4, 11, 11]
[13, 10, 6, 11, 13, 10, 8, 9, 14, 6]
[15, 9, 7, 7, 10, 9, 11, 12, 13, 7]
[11, 7, 13, 8, 9, 10, 13, 13, 6, 10]
[10, 5, 11, 6, 15, 14, 10, 9, 10, 10]
[4, 8, 10, 11, 7, 14, 14, 14, 7, 11]
[12, 11, 11, 11, 5, 12, 9, 8, 14, 7]
[14, 5, 7, 10, 11, 6, 14, 8, 20, 5]
[3, 12, 11, 8, 8, 7, 10, 14, 18, 9]
[9, 13, 1

KeyboardInterrupt: ignored

In [217]:
num_epochs = 8
from hw1.helper import Logger
import time, datetime
import pdb
logger = Logger(f'./logs/run_{3_53}/') 

# loss 
criterion = nn.CrossEntropyLoss()

step = 0

#find problem
model = net.to(device)


               
for i in range(num_epochs):
  for j,(images,labels)  in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
  
    #forward pass
    

    
    outputs  = model(images)

    loss  = criterion(outputs,labels).to(device) # pos can't be reverted because need preprocessing for outputs
    
    # backward pass
    optimizer.zero_grad()
  
    loss.backward()
 
    optimizer.step()
    
    
    # record the accuracy and loss
    step = step +1
   # pdb.set_trace()
  
    _,predicted = torch.max(outputs,1)
    accuracy  =  (labels == predicted).float().mean()
    info  = {'loss':loss.item(),'accuracy':accuracy.item()}
    if step %1000 ==0:
      for tag, value in info.items():
        logger.scalar_summary(tag,value,step)
        # 2. Log values and gradients of the parameters (histogram summary)
      for tag, value in model.named_parameters():
          tag = tag.replace('.', '/')
          logger.histo_summary(tag, value.data.cpu().numpy(), step)
          logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step)

      # 3. Log training images (image summary)
      info = { 'images': images.view(-1,3, 32, 32)[:10].cpu().numpy() }

      for tag, images in info.items():
          logger.image_summary(tag, images, step)

      #pdb.set_trace()
      print('accuracy:{}'.format(accuracy.item()))
      print('loss:{}'.format(loss.item()))

accuracy:0.3199999928474426
loss:1.8786665201187134
accuracy:0.4599999785423279
loss:1.5109987258911133
accuracy:0.4599999785423279
loss:1.6746816635131836
accuracy:0.41999998688697815
loss:1.5396859645843506


In [218]:
# check test accuracy
with torch.no_grad():
  correct = 0
  total  = 0
  for images,labels in test_loader:
    images= images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    #pdb.set_trace()
    _,predicted  = torch.max(outputs.data,1)
    correct +=  (labels == predicted).sum().item()
    total += labels.size(0)
    
  print('accuracy of the network for testing data is:{}'.format(correct/total))
    
    
    

accuracy of the network for testing data is:0.45078


### Q3.4 MNIST diagnosis 2 (10 pts)

*Hint:* Look at the test accuracy

**Diagnosis: Use the tensorboard to display the training images, it seems that the training data is selective which has a lot of zero than other data**

# New Section

In [123]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: http://9553b68e.ngrok.io


In [0]:
import torchvision
import torchvision.transforms as transforms
q_3_4_data_loader = dill.load( open( "q3_4_loader.p", "rb" ) )
# Load the MNIST dataset (images and labels, both train and test) into 2 DataLoaders
train_dataset = torchvision.datasets.MNIST(root='./data_mnist', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data_mnist', 
                                          train=False, 
                                          transform=transforms.ToTensor(),
                                          download = True)
# load data
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 100,shuffle =True)
test_loader  = torch.utils.data.DataLoader(test_dataset,batch_size = 100,shuffle =False)
q_3_4_net_loader = dill.load( open( "q3_4_net.p", "rb" ) )
net, optimizer = q_3_4_net_loader()


In [121]:
#problem found

num_epochs = 8
from hw1.helper import Logger
import time, datetime
import pdb
now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/run_{3.4}/') 

# loss 
criterion = nn.CrossEntropyLoss()

step = 0

# problem fix
model = net.to(device)


               
for i in range(num_epochs):
  for j,(images,labels)  in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    #pdb.set_trace()
    #forward pass
   
    #pdb.set_trace()
    outputs  = model(images)
    #pdb.set_trace()
    loss  = criterion(outputs,labels) # pos can't be reverted because need preprocessing for outputs
    
    # backward pass
    optimizer.zero_grad()
    #pdb.set_trace()
    loss.backward()
    #pdb.set_trace()
    optimizer.step()
    
    
    # record the accuracy and loss
    step = step +1
   # pdb.set_trace()
    _,predicted = torch.max(outputs,1)
    accuracy  =  (labels == predicted).float().mean()
    info  = {'loss':loss.item(),'accuracy':accuracy.item()}
    for tag, value in info.items():
      logger.scalar_summary(tag,value,step)
      # 2. Log values and gradients of the parameters (histogram summary)
    for tag, value in model.named_parameters():
        tag = tag.replace('.', '/')
        logger.histo_summary(tag, value.data.cpu().numpy(), step)
        logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step)

    # 3. Log training images (image summary)
    info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

    for tag, images in info.items():
        logger.image_summary(tag, images, step)
    if step %100 ==0:
      print('accuracy:{}'.format(accuracy.item()))
      print('loss:{}'.format(loss.item()))

accuracy:0.8399999737739563
loss:0.4967266917228699
accuracy:0.8999999761581421
loss:0.4229702353477478
accuracy:0.9599999785423279
loss:0.20248152315616608
accuracy:0.9399999976158142
loss:0.2676102817058563
accuracy:0.9399999976158142
loss:0.2403743863105774
accuracy:0.9599999785423279
loss:0.17210088670253754
accuracy:0.9099999666213989
loss:0.25721102952957153
accuracy:0.9399999976158142
loss:0.19654721021652222
accuracy:0.9300000071525574
loss:0.20346398651599884
accuracy:0.949999988079071
loss:0.12882602214813232
accuracy:0.9399999976158142
loss:0.22992286086082458
accuracy:0.9699999690055847
loss:0.10716312378644943
accuracy:0.9399999976158142
loss:0.11238694190979004
accuracy:0.9699999690055847
loss:0.1914239078760147
accuracy:0.9899999499320984
loss:0.02256031334400177
accuracy:0.9899999499320984
loss:0.0809394046664238
accuracy:0.9799999594688416
loss:0.10043692588806152
accuracy:0.949999988079071
loss:0.14007031917572021
accuracy:0.9799999594688416
loss:0.06992287933826447
a

In [122]:
# check test accuracy
with torch.no_grad():
  correct = 0
  total  = 0
  for images,labels in test_loader:
    images= images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    #pdb.set_trace()
    _,predicted  = torch.max(outputs.data,1)
    correct +=  (labels == predicted).sum().item()
    total += labels.size(0)
    
  print('accuracy of the network for testing data is:{}'.format(correct/total))
    
    

accuracy of the network for testing data is:0.9813


### Q3.5 CIFAR diagnosis 2 (10 pts)

**Diagnosis: the learning rate for the optimizer is 30000, which is too large**

In [193]:
q_3_5_data_loader = dill.load( open( "q3_5_loader.p", "rb" ) )
train_loader, test_loader = q_3_5_data_loader()
q_3_5_net_loader = dill.load( open( "q3_5_net.p", "rb" ) )
net,optimizer = q_3_5_net_loader()
model = net.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

Files already downloaded and verified
Files already downloaded and verified


In [176]:
num_epochs = 8
from hw1.helper import Logger
import time, datetime
import pdb
logger = Logger(f'./logs/run_{3_53}/') 

# loss 
criterion = nn.CrossEntropyLoss()

step = 0

#find problem
model = net.to(device)


               
for i in range(num_epochs):
  for j,(images,labels)  in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
  
    #forward pass
    

    
    outputs  = model(images)

    loss  = criterion(outputs,labels).to(device) # pos can't be reverted because need preprocessing for outputs
    
    # backward pass
    optimizer.zero_grad()
  
    loss.backward()
 
    optimizer.step()
    
    
    # record the accuracy and loss
    step = step +1
   # pdb.set_trace()
  
    _,predicted = torch.max(outputs,1)
    accuracy  =  (labels == predicted).float().mean()
    info  = {'loss':loss.item(),'accuracy':accuracy.item()}
    if step %1000 ==0:
      for tag, value in info.items():
        logger.scalar_summary(tag,value,step)
        # 2. Log values and gradients of the parameters (histogram summary)
      for tag, value in model.named_parameters():
          tag = tag.replace('.', '/')
          logger.histo_summary(tag, value.data.cpu().numpy(), step)
          logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step)

      # 3. Log training images (image summary)
      info = { 'images': images.view(-1,3, 32, 32)[:10].cpu().numpy() }

      for tag, images in info.items():
          logger.image_summary(tag, images, step)

      #pdb.set_trace()
      print('accuracy:{}'.format(accuracy.item()))
      print('loss:{}'.format(loss.item()))

accuracy:0.3499999940395355
loss:1.7990939617156982
accuracy:0.38999998569488525
loss:1.6045722961425781
accuracy:0.3999999761581421
loss:1.7973638772964478
accuracy:0.4899999797344208
loss:1.507317304611206


In [177]:
# check test accuracy
with torch.no_grad():
  correct = 0
  total  = 0
  net = net.to(device)
  for images,labels in test_loader:
    images= images.to(device)
    labels = labels.to(device)
    outputs = net(images)
    #pdb.set_trace()
    _,predicted  = torch.max(outputs.data,1)
    correct +=  (labels == predicted).sum().item()
    total += labels.size(0)
    
  print('accuracy of the network for testing data is:{}'.format(correct/total))

accuracy of the network for testing data is:0.4311


### Q3.6 CIFAR diagnosis 3 (10 pts)

**Diagnosis: The network is too deep which has more than 20 layers,which may be too much for this dataset and will bring overfitting problem. To fix it , I change the network structure the same as Q3.5**

In [189]:
import torch
q_3_6_data_loader = dill.load( open( "q3_6_loader.p", "rb" ) )
train_loader, test_loader = q_3_6_data_loader()

q_3_5_net_loader = dill.load( open( "q3_5_net.p", "rb" ) )
net,_ = q_3_5_net_loader()
model = net.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

Files already downloaded and verified
Files already downloaded and verified


In [187]:
print(net)

Q_3_5_net(
  (fc1): Linear(in_features=3072, out_features=8192, bias=True)
  (fc2): Linear(in_features=8192, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=2048, bias=True)
  (fc4): Linear(in_features=2048, out_features=1024, bias=True)
  (fc5): Linear(in_features=1024, out_features=10, bias=True)
)


In [190]:
num_epochs = 8
from hw1.helper import Logger
import time, datetime
import pdb
logger = Logger(f'./logs/run_{3_53}/') 

# loss 
criterion = nn.CrossEntropyLoss()

step = 0

#find problem
model = net.to(device)


               
for i in range(num_epochs):
  for j,(images,labels)  in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
  
    #forward pass
    

    
    outputs  = model(images)

    loss  = criterion(outputs,labels).to(device) # pos can't be reverted because need preprocessing for outputs
    
    # backward pass
    optimizer.zero_grad()
  
    loss.backward()
 
    optimizer.step()
    
    
    # record the accuracy and loss
    step = step +1
   # pdb.set_trace()
  
    _,predicted = torch.max(outputs,1)
    accuracy  =  (labels == predicted).float().mean()
    info  = {'loss':loss.item(),'accuracy':accuracy.item()}
    if step %1000 ==0:
      for tag, value in info.items():
        logger.scalar_summary(tag,value,step)
        # 2. Log values and gradients of the parameters (histogram summary)
      for tag, value in model.named_parameters():
          tag = tag.replace('.', '/')
          logger.histo_summary(tag, value.data.cpu().numpy(), step)
          logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step)

      # 3. Log training images (image summary)
      info = { 'images': images.view(-1,3, 32, 32)[:10].cpu().numpy() }

      for tag, images in info.items():
          logger.image_summary(tag, images, step)

      #pdb.set_trace()
      print('accuracy:{}'.format(accuracy.item()))
      print('loss:{}'.format(loss.item()))

accuracy:0.32999998331069946
loss:1.8425071239471436
accuracy:0.5299999713897705
loss:1.4569816589355469
accuracy:0.5099999904632568
loss:1.4345391988754272
accuracy:0.5399999618530273
loss:1.3951600790023804


In [191]:
# check test accuracy
with torch.no_grad():
  correct = 0
  total  = 0
  net = net.to(device)
  for images,labels in test_loader:
    images= images.to(device)
    labels = labels.to(device)
    outputs = net(images)
    #pdb.set_trace()
    _,predicted  = torch.max(outputs.data,1)
    correct +=  (labels == predicted).sum().item()
    total += labels.size(0)
    
  print('accuracy of the network for testing data is:{}'.format(correct/total))

accuracy of the network for testing data is:0.4799


### Q 3.7 MNIST diagnosis 3 (10 pts)

**Diagnosis: It seems that the labels for the dataset is wrong. To fix it, we change the dataset to the dataset in Q3.2.**

In [0]:
q_3_7_data_loader = dill.load( open( "q3_2_loader.p", "rb" ) )
train_loader, test_loader = q_3_7_data_loader()
q_3_7_net_loader = dill.load( open( "q3_7_net.p", "rb" ) )
net, optimizer = q_3_7_net_loader()

In [148]:
print(net)

Q_3_7_net(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=10, bias=True)
)


In [162]:
#problem found

num_epochs = 8
from hw1.helper import Logger
import time, datetime
import pdb
now = time.mktime(datetime.datetime.now().timetuple())
logger = Logger(f'./logs/run_{3.72}/') 

# loss 
criterion = nn.CrossEntropyLoss()

step = 0

# problem fix
model = net.to(device)


               
for i in range(num_epochs):
  for j,(images,labels)  in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    #pdb.set_trace()
    #forward pass
   
    #pdb.set_trace()
    outputs  = model(images)
    #pdb.set_trace()
    loss  = criterion(outputs,labels) # pos can't be reverted because need preprocessing for outputs
    
    # backward pass
    optimizer.zero_grad()
    #pdb.set_trace()
    loss.backward()
    #pdb.set_trace()
    optimizer.step()
    
    
    # record the accuracy and loss
    step = step +1
   # pdb.set_trace()
    _,predicted = torch.max(outputs,1)
    accuracy  =  (labels == predicted).float().mean()
    info  = {'loss':loss.item(),'accuracy':accuracy.item()}
    for tag, value in info.items():
      logger.scalar_summary(tag,value,step)
      # 2. Log values and gradients of the parameters (histogram summary)
    for tag, value in model.named_parameters():
        tag = tag.replace('.', '/')
        logger.histo_summary(tag, value.data.cpu().numpy(), step)
        logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step)
    #pdb.set_trace()
    # 3. Log training images (image summary)
    info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

    for tag, images in info.items():
        logger.image_summary(tag, images, step)
        
    for i in range(10):
      
      logger.scalar_summary('label:'+str(i),labels[i].item(),step)
    

    if step %100 ==0:
      print('accuracy:{}'.format(accuracy.item()))
      print('loss:{}'.format(loss.item()))

accuracy:0.8700000047683716
loss:0.4446192681789398
accuracy:0.9799999594688416
loss:0.21155518293380737
accuracy:0.9599999785423279
loss:0.1697012186050415
accuracy:0.9699999690055847
loss:0.13947707414627075
accuracy:0.9899999499320984
loss:0.05399828031659126


In [163]:
# check test accuracy
with torch.no_grad():
  correct = 0
  total  = 0
  net = net.to(device)
  for images,labels in test_loader:
    images= images.to(device)
    labels = labels.to(device)
    outputs = net(images)
    #pdb.set_trace()
    _,predicted  = torch.max(outputs.data,1)
    correct +=  (labels == predicted).sum().item()
    total += labels.size(0)
    
  print('accuracy of the network for testing data is:{}'.format(correct/total))

accuracy of the network for testing data is:0.922
